### Given a recording, find any frames that have been skipped 

- there are two types of dropped frames (1) frames that are correctly skipped in `frame_id` and (2) frames that are not correctly skipped in `frame_id`

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [20]:
metadata_file = 'example_csvs/example.flir.metadata.csv'

In [21]:
metadata_df = pd.read_csv(metadata_file)

In [22]:
metadata_df[:3]

,frame_id,frame_timestamp,frame_image_uid
0,1,3603480856728,1.673547e+09
1,2,3603514161960,1.673547e+09
2,3,3603547468960,1.673547e+09


### Make a corrupted dataframe
- the example dataframe has no dropped frames, so we can make an example corrupted dataframe

In [23]:
# rows 46 and 51 will have dropped frames which are not correctly labeled as dropped, whereas 24 and 100 are 
corrupted_metadata_df = metadata_df.drop([46, 51,52])
corrupted_metadata_df['frame_id'] = np.arange(len(corrupted_metadata_df.values))+1
corrupted_metadata_df = corrupted_metadata_df.drop([24, 100])
corrupted_metadata_df.to_csv(
    "example_csvs/example.corrupted.flir.metadata.csv", index=False
)

### Correct the dataframe

In [24]:
corrupted_metadata_df = pd.read_csv("example_csvs/example.corrupted.flir.metadata.csv")

In [25]:
from pathlib2 import Path
def correct_corrupted_metadata(csv_loc, framerate = 30, save_result = True, save_prefix = 'corrected.'):
    # load data
    csv_loc = Path(csv_loc)
    corrupted_metadata_df = pd.read_csv(csv_loc)
    # compute the skipped frames
    skipped_frames = np.diff(corrupted_metadata_df.frame_id.values)
    
    print('Dataset contains {} known skipped frames'.format(np.sum(skipped_frames -1)))
    
    # compute the expected delta between frames
    expected_delta_between_frames_s = 1/framerate
    expected_delta_between_frames_ns = expected_delta_between_frames_s * 1e9
    # compute the actual delta between frames 
    framerate_delta = np.diff(corrupted_metadata_df.frame_timestamp.values) / expected_delta_between_frames_ns
    # compute the actual delta between frames relative to the expected delta
    frames_offset = np.round(framerate_delta - skipped_frames).astype(int)
    n_errors = np.sum(frames_offset > 0)
    if n_errors > 0: 
        print(
            "Found {} additional skipped frames. Correcting.".format(n_errors)
        )
        # fix the frame numbers
        corrupted_metadata_df["frame_id"] = corrupted_metadata_df["frame_id"]
        corrupted_metadata_df["frame_id"] = [1] + list(
            corrupted_metadata_df.frame_id[1:].values + np.cumsum(frames_offset)
        )
        if save_result:
            save_loc = Path(csv_loc).parent / (save_prefix + Path(csv_loc).name)
            corrupted_metadata_df.to_csv(
                save_loc, index=False
            )
        else:
            return corrupted_metadata_df
    else:
        print('No additional errors found.')

In [27]:
correct_corrupted_metadata("example_csvs/example.corrupted.flir.metadata.csv")

Dataset contains 2 known skipped frames
Found 2 additional skipped frames. Correcting.


In [28]:
correct_corrupted_metadata("example_csvs/corrected.example.corrupted.flir.metadata.csv")

Dataset contains 5 known skipped frames
No additional errors found.
